In [1]:
%run QM9GNN2_Multitask.ipynb

Using TensorFlow backend.
/home/sohompaul/psi4conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sohompaul/psi4conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sohompaul/psi4conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sohompaul/psi4conda/lib/python3.6/site-pa

Loading QM9 dataset.
Reading SDF


100%|██████████| 133885/133885 [01:18<00:00, 1708.58it/s]


In [2]:
import psi4
import time
from os import path, getcwd

In [3]:
def predict_property_neural(prop=None, mol_id=-1):
    if mol_id == -1:
        raise ValueError("ID must be between 1 and 133885")
    for cluster in clusters:
        if prop in cluster:
            model = create_hard_parameter_sharing_model(len(cluster))
            model.load_weights(generate_model_filename(cluster))
            model.compile(optimizer='adam', loss='mse')
            predictions = model.predict([[X_complete[mol_id-1]], [A_complete[mol_id-1]], [E_complete[mol_id-1]]])
            mean, std = 0, 1
            with open(generate_model_helper_filename(prop), 'r') as f:
                lines = f.readlines()
                mean = float(lines[0].strip())
                std = float(lines[1].strip())
            prediction = mean + std * predictions[cluster.index(prop)]
            return prediction[0][0]
    if prop == 'gap':
        lumo = predict_property_neural(prop='lumo', mol_id=mol_id)
        homo = predict_property_neural(prop='homo', mol_id=mol_id)
        return lumo - homo
    raise ValueError("Property was not found in clusters list")

In [4]:
def get_data_folder_path():
    return path.join(getcwd(), '..', '..', 'dsgdb9nsd')

In [5]:
def get_molecule_from_file(filenum):
    filepath = path.join(get_data_folder_path(), 
                           'dsgdb9nsd_' + str(filenum).zfill(6) + '.xyz')
    f = open(filepath, 'r')
    lines = f.readlines()
    f.close()
    num_atoms = int(lines[0])
    atom_list = lines[2:2+num_atoms]
    for i in range(len(atom_list)):
        atom_list[i] = atom_list[i][:atom_list[i].rfind("\t")] + "\n"
    return psi4.geometry("".join(atom_list))

In [6]:
def generate_dft_output_file_path(filenum):
    return path.join('psi4_output', 'output_'+str(filenum)+'.dat')

In [7]:
def process_molecule(filenum, thermochemical=False):
    psi4.core.set_output_file(generate_dft_output_file_path(filenum), False)
    psi4.set_memory('2 GB')
    molecule = get_molecule_from_file(filenum)
    if thermochemical:
        e, wfn = psi4.freq('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    else:
        e, wfn = psi4.energy('b3lyp/cc-pvqz', molecule=molecule, return_wfn=True)
    return wfn

In [8]:
def extract_rotational_constants(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Rotational constants:') > -1 and lines[i].find('[MHz]') > -1:
            words = lines[i].split()
            rot_constants = []
            for const in [words[4], words[7], words[10]]:     
                if const.isnumeric():
                    rot_constants.append(float(const)/1000)
                else:
                    rot_constants.append(const)
                    # rot_constants.append(float(const)/1000)
            return rot_constants
    return None, None, None

In [9]:
def extract_dipole_moment(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Dipole Moment: [D]") > -1:
            return lines[i+1][lines[i+1].find("Total:") + 6:]

In [10]:
def extract_homo_lumo(filenum, wfn):
    homo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha())
    lumo = wfn.epsilon_a_subset("AO", "ALL").get(wfn.nalpha() + 1)
    return homo, lumo

In [11]:
def extract_gap(filenum, wfn):
    homo, lumo = extract_homo_lumo_gap(fileum, wfn)
    return lumo - homo

In [12]:
def extract_zpve(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total ZPE, Electronic energy at 0 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [13]:
def extract_zero_point_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E0, Electronic energy") > -1:
            words = lines[i].split()
            return words[-2]

In [14]:
def extract_internal_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total E, Electronic energy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [15]:
def extract_enthalpy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total H, Enthalpy at  298.15 [K]") > -1:
            words = lines[i].split()
            return words[-2]

In [16]:
def extract_gibbs_free_energy(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find("Total G,") > -1:
            words = lines[i].split()
            return words[-2]

In [17]:
def extract_cv(filenum, wfn):
    f = open(generate_dft_output_file_path(filenum), 'r')
    lines = f.readlines()
    f.close()
    for i in range(len(lines)-1, -1, -1):
        if lines[i].find('Total Cv') > -1:
            words = lines[i].split()
            return words[2]

In [18]:
def batch_process(start_num, end_num, thermochemical=False):
    f = open("output.csv", "w")
    output_header = "Index,A,B,C,Dipole,HOMO,LUMO"
    if thermochemical:
        output_header += ",zpve,H 298.15,G 298.15"
    output_header += "\n"
    f.write(output_header)
    for filenum in range(start_num, end_num+1):
        wfn = process_molecule(filenum, thermochemical=thermochemical)
        a, b, c = extract_rotational_constants(filenum, wfn)
        dipole = extract_dipole_moment(filenum, wfn)
        homo, lumo = extract_homo_lumo(filenum, wfn)
        output = str(filenum) + "," + str(a) + "," + str(b) + "," + str(c) + "," + str(dipole) + "," + str(homo) + "," + str(lumo)
        if thermochemical:
            zpve = extract_zpve(filenum, wfn)
            enthalpy = extract_enthalpy(filenum, wfn)
            gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
            output += "," + str(zpve) + "," + str(enthalpy) + "," + str(gibbs_free_energy)
        output += "\n" 
        f.write(output)
    f.close()

In [19]:
def predict_all_properties_dft(filenum=-1, thermochemical=False):
    if filenum == -1:
        raise ValueError("Filenum must be between 1 and 133885")
    wfn = process_molecule(filenum, thermochemical=thermochemical)
    a, b, c = extract_rotational_constants(filenum, wfn)
    dipole = extract_dipole_moment(filenum, wfn)
    homo, lumo = extract_homo_lumo(filenum, wfn)
    ret_dict = {'A': a, 'B': b, 'C': c, 'mu': dipole, 'homo': homo, 
                'lumo': lumo, 'gap': lumo-homo}
    if thermochemical:
        zpve = extract_zpve(filenum, wfn)
        internal_energy = extract_internal_energy(filenum, wfn)
        u0 = extract_zero_point_internal_energy(filenum, wfn)
        enthalpy = extract_enthalpy(filenum, wfn)
        gibbs_free_energy = extract_gibbs_free_energy(filenum, wfn)
        cv = extract_cv(filenum, wfn)
        ret_dict.update({'zpve': zpve, 'u0': u0, 'u298': internal_energy, 
                           'h298': enthalpy, 'g298': gibbs_free_energy, 'cv': cv})
    return ret_dict

In [20]:
def lookup_property(prop=None, mol_id=-1):
    return y_complete.loc[mol_id-1, prop]

In [21]:
def prompt_user_for_calculation():
    while True:
        num = -1
        while num < 1 or num > 133885:
            try:
                num = int(input('Choose a molecule index (1-133885): '))
            except ValueError:
                print("Please provide a valid number")
                num = -1
            
        properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']
        prop = None
        while prop not in properties:
            print("Choose an available property from the following:")
            print("A, B, C, mu, homo, lumo, gap, zpve, u0, u298, h298, g298, cv")
            prop = input('Choose a property: ')
        
        dft = None
        while dft not in ['0', '1']:
            print("Choose whether to use DFT or neural methods")
            print("0 for neural methods, 1 for DFT")
            dft = input('Calculation type: ')
        
        if dft == '1':
            print('Beginning DFT calculation')
            start = time.time()
            thermochemical = prop in properties[7:]
            ret_dict = predict_all_properties_dft(num, thermochemical=thermochemical)
            print(ret_dict[prop])
            end = time.time()
            print('DFT calculation took', end-start, 's')
        else:
            print('Beginning neural calculation:')
            start = time.time()
            print(predict_property_neural(prop=prop, mol_id=num))
            end = time.time()
            print('Neural method took', end-start, 's')
        print('Actual data:')
        print(lookup_property(prop=prop, mol_id=num))

In [23]:
# prompt_user_for_calculation()

In [24]:
properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']

for prop in properties:
    print(prop)
    errors = list()
    for index in np.random.choice(133885, 10):
        pred = predict_property_neural(prop=prop, mol_id=index+1)
        actual = lookup_property(prop=prop, mol_id=index+1)
        err = abs((pred-actual)/actual*100)
        print(err)
        errors.append(err)
    print('total err', sum(errors)/len(errors))

A




13.974896915369206
19.67161963262741
20.356241798183174
0.45548724303628735
12.295963724982977
33.739341069475174
7.333896654863099
16.2314518401516
11.238227112691032
7.984859416456194
total err 14.328198540783614
B
11.660992977053997
9.327389734717602
44.647606070950026
0.18840437339345034
2.044627647050733
16.706771919304504
89.07799979478824
28.88464143613837
19.511916797854326
1.9935638651811403
total err 22.404391461643236
C
17.593401820816425
4.460386066406172
21.06142246990432
1.547464977332565
16.088114003414802
0.028043254895753416
16.702896364275112
4.754930128877823
10.004258214313836
48.46520995602857
total err 14.070612725626537
mu
11.367001830452109
14.012089122300608
16.4974316858694
43.94863955650455
21.00295915601695
8.229106580707398
15.253320982465816
23.13965831622461
11.937425335573481
3.10102382345422
total err 16.848865638956916
homo
7.80007422104915
5.406640082751616
0.2884830972800146
3.7183623079519723
0.42195105853242
3.0216451950572254
1.7655888748661

KeyboardInterrupt: 

In [ ]:
properties = ['A', 'B', 'C', 'mu', 'homo', 'lumo', 'gap', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv']

for prop in properties:
    print(prop)
    errors = list()
    for index in np.random.choice(133885, 10):
        pred = predict_property_neural(prop=prop, mol_id=index+1)
        actual = lookup_property(prop=prop, mol_id=index+1)
        err = abs(pred-actual)
        print(err)
        errors.append(err)
    print('avg err', sum(errors)/len(errors))

A
1.8455707850646963
0.718402177734375
0.048991450347900756
0.04634370468139659
0.16163083320617666
0.6746652793884276
0.747865013427734
0.9222380640411378
0.44993365722656264
0.1536929026794427
avg err 0.576933386779785
B
0.6436539069366456
0.2824262729263307
0.04712434226989748
0.1802233409881593
0.03706696678161614
0.09517442649841312
0.13754784400939934
0.28446955924987793
0.2863973701667786
0.41041322746276854
avg err 0.24044972572898865
C
0.16003034473419198
0.0962758953857421
0.19725141418457026
0.05663787128448483
0.009670214366912844
0.10894170333862307
0.10201684257507337
0.009325303211212166
0.05882413249969476
0.0353913743972778
avg err 0.08343650959777832
mu
2.3182340900421146
0.3333273456573487
1.2832710319519043
1.3116816108703615
0.11640046367645263
0.1574854789733886
1.5667521926879884
0.698894987106323
0.22679478683471688
1.0718364540100098
avg err 0.9084678441810607
homo
0.006094772720336927
0.004701627635955802
0.0026826119422912464
4.733912944793195e-05
0.015053135